In [35]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='1'
os.environ['TF_ENABLE_MKL_NATIVE_FORMAT']='1'
os.environ['KMP_BLOCKTIME']='1'
os.environ['KMP_SETTINGS']='1'
os.environ['OMP_NUM_THREADS']='6'
os.environ['KMP_AFFINITY']='granularity=fine,verbose,compact,1,0'

In [36]:
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(2) 
tf.config.threading.set_intra_op_parallelism_threads(8)
tf.config.set_soft_device_placement(False)

def get_mkl_enabled_flag():

    mkl_enabled = False
    major_version = int(tf.__version__.split(".")[0])
    minor_version = int(tf.__version__.split(".")[1])
    if major_version >= 2:
        if minor_version < 5:
            from tensorflow.python import _pywrap_util_port
        elif minor_version >= 9:

            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '1'))

        else:
            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '0'))
        mkl_enabled = _pywrap_util_port.IsMklEnabled() or (onednn_enabled == 1)
    else:
        mkl_enabled = tf.pywrap_tensorflow.IsMklEnabled()
    return mkl_enabled

print ("We are using Tensorflow version", tf.__version__)
print("MKL enabled :", get_mkl_enabled_flag())

We are using Tensorflow version 2.12.0
MKL enabled : True


In [37]:
import numpy as np # linear algebra
import sklearnex
from sklearnex import patch_sklearn
patch_sklearn() 
import modin.pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from time import time

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [38]:
import json

with open(r'./dataset/intents.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])

In [39]:
df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
76,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,gibberish,[jhjbdsjcxn],"[I'm sorry, I couldn't understand your message..."
79,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [40]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

In [41]:
df = pd.DataFrame.from_dict(dic)

In [42]:
df

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
228,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
229,gibberish,jhjbdsjcxn,"[I'm sorry, I couldn't understand your message..."
230,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
231,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


In [43]:
df.to_csv('./dataset/dataframe_chatbot.csv', index=True)

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))


number of unique words =  304
X shape =  (233, 18)
y shape =  (233,)
num of classes =  81


In [45]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()
plot_model(model, show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 18, 100)           30500     
                                                                 
 lstm_6 (LSTM)               (None, 18, 32)            17024     
                                                                 
 layer_normalization_10 (Lay  (None, 18, 32)           64        
 erNormalization)                                                
                                                                 
 lstm_7 (LSTM)               (None, 18, 32)            8320      
                                                                 
 layer_normalization_11 (Lay  (None, 18, 32)           64        
 erNormalization)                                                
                                                                 
 lstm_8 (LSTM)               (None, 32)               

In [46]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=30)

Epoch 1/30
24/24 [==============================] - 9s 16ms/step - loss: 5.0407 - accuracy: 0.0172
Epoch 2/30
24/24 [==============================] - 0s 17ms/step - loss: 3.9725 - accuracy: 0.0944
Epoch 3/30
24/24 [==============================] - 0s 17ms/step - loss: 3.3273 - accuracy: 0.1845
Epoch 4/30
24/24 [==============================] - 0s 17ms/step - loss: 2.7689 - accuracy: 0.3562
Epoch 5/30
24/24 [==============================] - 0s 17ms/step - loss: 2.2700 - accuracy: 0.4850
Epoch 6/30
24/24 [==============================] - 0s 18ms/step - loss: 1.8939 - accuracy: 0.5794
Epoch 7/30
24/24 [==============================] - 0s 17ms/step - loss: 1.5469 - accuracy: 0.6824
Epoch 8/30
24/24 [==============================] - 0s 17ms/step - loss: 1.2785 - accuracy: 0.7339
Epoch 9/30
24/24 [==============================] - 0s 17ms/step - loss: 1.0185 - accuracy: 0.7983
Epoch 10/30
24/24 [==============================] - 0s 17ms/step - loss: 0.8449 - accuracy: 0.8841
Epoch 11/

In [47]:
import re
import random

def model_responce(query, model):
        text = []
        txt = re.sub('[^a-zA-Z\']', ' ', query)
        txt = txt.lower()
        txt = txt.split()
        txt = " ".join(txt)
        text.append(txt)
        x_test = tokenizer.texts_to_sequences(text)
        if (len(query.split(' ')) > 1):
                x_test = np.array(x_test).squeeze()
        else:
                x_test = np.reshape(x_test, -1)
        try:
                x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
        except Exception as e:
                return "Error 404: Word not found in my dictionary"
        y_pred = model.predict(x_test)
        y_pred = y_pred.argmax()
        tag = lbl_enc.inverse_transform([y_pred])[0]
        responses = df[df['tag'] == tag]['responses'].values[0]

        print("you: {}".format(query))
        print("model: {}".format(random.choice(responses)))


In [48]:
model_responce('Hi! How are you?', model)

1/1 [==============================] - 3s 3s/step
you: Hi! How are you?
model: Hi there. What brings you here today?


In [49]:
model_responce('How can I maintain social connections? What if I feel lonely?', model)

1/1 [==============================] - 0s 21ms/step
you: How can I maintain social connections? What if I feel lonely?
model: A lot of people are alone right now, but we don't have to be lonely. We're all in this together. Think about the different ways to connect that are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one set time for a video call. Then, work with your social networks to make a plan. You might video chat with your close friends in the evening and phone a family member once a week. Remember to be mindful of people who may not be online. Check in by phone and ask how you can help. The quality of your social connections matter. Mindlessly scrolling through social media and liking a few posts usually doesn't build strong social connections. Make sure you focus on strategies that actually make you feel included and connected. If your current strategies don't help you feel con

In [50]:
model_responce('Well... I\'m feeling sad!', model)

1/1 [==============================] - 0s 37ms/step
you: Well... I'm feeling sad!
model: Hello there. Glad to see you're back. What's going on in your world right now?


In [51]:
model_responce('Maybe I just didn\'t want to be born :)', model)

1/1 [==============================] - 0s 22ms/step
you: Maybe I just didn't want to be born :)
model: I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821.


In [52]:
model_responce('I want commit suicide', model)

1/1 [==============================] - 0s 19ms/step
you: I want commit suicide
model: I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821.


In [53]:
model_responce('I am facing a lot of problems', model)

1/1 [==============================] - 0s 19ms/step
you: I am facing a lot of problems
model: Nice to meet you. So tell me. How do you feel today?


In [54]:
model_responce('i do not think you can help me', model)

1/1 [==============================] - 0s 23ms/step
you: i do not think you can help me
model: Nice to meet you. So tell me. How do you feel today?


In [55]:
model_responce('I need help', model)

1/1 [==============================] - 0s 31ms/step
you: I need help
model: Sure. Tell me how can i assist you


In [56]:
model_responce('Good morning', model)

1/1 [==============================] - 0s 25ms/step
you: Good morning
model: Good morning. I hope you had a good night's sleep. How are you feeling today? 


In [57]:
model_responce('Good evening', model)

1/1 [==============================] - 0s 23ms/step
you: Good evening
model: Good evening. How has your day been?


In [60]:
model_responce('Gosadj kjasndkja sadan  kadsjnbkjabdhjk', model)

1/1 [==============================] - 0s 20ms/step
you: Gosadj kjasndkja sadan  kadsjnbkjabdhjk
model: Not sure I understand that.


In [59]:
model.save('./chatbot_model.h5', model)